<a href="https://colab.research.google.com/github/tylerjmwhit/Maching_learning_pratice_notebooks/blob/main/week8lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.datasets.mnist import load_data
import tensorflow as tf

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
(train_x, train_y), (test_x, test_y) = load_data() #loading MNIST DATA

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# my rotations
train_x_90 = np.rot90(train_x[15000:45000],axes=(1,2))    #rotating some 90
train_x_180 = np.rot90(train_x[:30000],2,axes=(1,2)) #rotating some 180
train_x_270 = np.rot90(train_x[30000:],3,axes=(1,2)) #rotating some 270

In [ ]:
# Putting all arrays together into data set
train_x_expanded = []
train_x_expanded = np.concatenate((train_x,train_x_90,train_x_180,train_x_270))

In [ ]:
#now to make the train_y data set match up
train_y_90 = train_y[15000:45000]   #rotating some 90
train_y_180 = train_y[:30000] #rotating some 180
train_y_270 = train_y[30000:] #rotating some 270

In [ ]:
train_y_expanded = []
train_y_expanded = np.concatenate((train_y,train_y_90,train_y_180,train_y_270))
train_x_expanded = train_x_expanded.reshape(-1,784)
print(train_x_expanded.shape,train_y_expanded.shape)


(150000, 784) (150000,)


In [ ]:
# Function to create model, required for KerasClassifier
def create_model(first_hidden_layer_size, second_hidden_layer_size, solver, act):
	# create model
	model = Sequential()
	model.add(Dense(first_hidden_layer_size, input_shape=(784,), activation=act))

	model.add(Dense(second_hidden_layer_size, activation=act))
	model.add(Dense(10, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer=solver, metrics=['accuracy'])
	return model

classifier = KerasClassifier(build_fn=create_model)

<ipython-input-8-7624e4f57e05>:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=create_model)


In [ ]:
from tensorflow.keras.utils import to_categorical
param_grid = {
    "first_hidden_layer_size": [10, 20, 30],
    "second_hidden_layer_size": [20, 30, 40],
    "act":["relu","sigmoid","tanh"],
    "solver": ['RMSprop', 'sgd', 'adam']
}
gs = RandomizedSearchCV(classifier, param_grid, cv = 3, verbose = 2,  )
gs.fit(train_x_expanded,to_categorical(train_y_expanded))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
1563/1563 [==============================] - 3s 2ms/step - loss: 1.7011 - accuracy: 0.3523
[CV] END act=sigmoid, first_hidden_layer_size=20, second_hidden_layer_size=20, solver=adam; total time=  23.4s
1563/1563 [==============================] - 3s 2ms/step - loss: 1.9158 - accuracy: 0.3637
[CV] END act=sigmoid, first_hidden_layer_size=20, second_hidden_layer_size=20, solver=adam; total time=  16.1s
1563/1563 [==============================] - 3s 2ms/step - loss: 2.1831 - accuracy: 0.2508
[CV] END act=sigmoid, first_hidden_layer_size=20, second_hidden_layer_size=20, solver=adam; total time=  13.7s
1563/1563 [==============================] - 3s 2ms/step - loss: 1.6468 - accuracy: 0.3967
[CV] END act=tanh, first_hidden_layer_size=30, second_hidden_layer_size=20, solver=RMSprop; total time=  13.7s
1355/1563 [=========================>....] - ETA: 0s - loss: 2.1345 - accuracy: 0.3571

In [ ]:
df = pd.DataFrame(gs.cv_results_['params'])
df['mean_test_score'] = gs.cv_results_['mean_test_score']
df

,solver,second_hidden_layer_size,first_hidden_layer_size,act,mean_test_score
0,sgd,20,20,relu,0.124660
1,RMSprop,30,10,sigmoid,0.321667
2,adam,20,10,relu,0.195220
3,adam,40,30,tanh,0.342553
4,adam,30,30,sigmoid,0.327133
5,adam,40,30,sigmoid,0.324613
6,sgd,30,20,tanh,0.297713
7,adam,30,10,tanh,0.298327
8,sgd,30,30,relu,0.112807
9,sgd,30,20,sigmoid,0.326747
